# Load the Population Receptive Field properties for the Connective Fields

In [1]:
import pandas as pd
import numpy as np
import os 
import pickle 
from nibabel.freesurfer.io import read_morph_data, write_morph_data
import io

In [2]:
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives'
freesurfer = f"{MAIN_PATH}/freesurfer"
source = 'V1'
denoising = 'nordic'
atlases = ['benson', 'manual']
subjects = [f'sub-{i:02}' for i in range(2, 47)]
tasks = ['RET2','RET']
#tasks = ['RestingState']
hemispheres = ['lh', 'rh']
visual_areas = ['V1', 'V2', 'V3']
analysis = 'benson total'  # or 'benson total' or 'standard

In [3]:
class PRFModel:
    def __init__(self, r2, size, ecc, angle):
        self.r2 = r2        
        self.size = size
        self.ecc = ecc    
        self.angle = angle     

def pickle_file(filepath):
    if not os.path.isfile(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")
    with open(filepath, 'rb') as file: 
        return pickle.load(file)

def load_prf(subj, main_path, atlas, denoising, task):
    filename_task = 'RET' if task == 'RestingState' else task
    filepath = os.path.join(
        main_path,
        f'pRFM/{subj}/ses-02/{denoising}/model-{atlas}-nelder-mead-GM_desc-prf_params_{filename_task}.pkl')
    pkl_data = pickle_file(filepath)
    prf_params = pkl_data['model'].iterative_search_params     
    prf_voxels = np.where(pkl_data['rois_mask'] == 1)[0]
    return prf_params, prf_voxels

def filter_prf(prf_params, prf_voxels):
    return PRFModel(
        r2=prf_params[:, 7],
        size=prf_params[:, 2],
        ecc=np.sqrt(prf_params[:, 1]**2 + prf_params[:, 0]**2),
        angle=np.arctan2(prf_params[:, 1], prf_params[:, 0]))

def load_label_data(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    lines = lines[2:]  # skip header
    data = [line.strip().split() for line in lines]
    indices = [int(row[0]) for row in data]
    values = [float(row[4]) for row in data]  # value stored in 5th column
    return dict(zip(indices, values))

def extract_prf_previous(subj, hemi, target, csv_path, prf_params, prf_voxels):
    best_fit = pd.read_csv(csv_path)

    if 'Target Vertex Index' in best_fit.columns:
        repeated_header_rows = best_fit['Target Vertex Index'] == 'Target Vertex Index'
        if repeated_header_rows.any():
            best_fit = best_fit[~repeated_header_rows]

    best_fit['Source Vertex Index'] = best_fit['Source Vertex Index'].astype(int)
    best_fit['Target Vertex Index'] = best_fit['Target Vertex Index'].astype(int)

    prf_model = filter_prf(prf_params, prf_voxels)
    lh_c = read_morph_data(f'{freesurfer}/{subj}/surf/lh.curv')
    numel_lh = lh_c.shape[0]

    if hemi == 'rh':
        prf_voxels_rh = prf_voxels[prf_voxels >= numel_lh]
        prf_voxels_rh_adjusted = prf_voxels_rh - numel_lh
        prf_ecc_rh = prf_model.ecc[prf_voxels >= numel_lh]
        prf_angle_rh = prf_model.angle[prf_voxels >= numel_lh]
        voxel_to_ecc = dict(zip(prf_voxels_rh_adjusted, prf_ecc_rh))
        voxel_to_angle = dict(zip(prf_voxels_rh_adjusted, prf_angle_rh))
    else:
        voxel_to_ecc = dict(zip(prf_voxels, prf_model.ecc))
        voxel_to_angle = dict(zip(prf_voxels, prf_model.angle))

    best_fit['Source Eccentricity'] = best_fit['Source Vertex Index'].map(voxel_to_ecc)
    best_fit['Source Polar Angle'] = best_fit['Source Vertex Index'].map(voxel_to_angle)
    return best_fit

def extract_prf(subj, hemi, target, csv_path, prf_params, prf_voxels):
    best_fit = pd.read_csv(csv_path)

    if 'Target Vertex Index' in best_fit.columns:
        repeated_header_rows = best_fit['Target Vertex Index'] == 'Target Vertex Index'
        if repeated_header_rows.any():
            best_fit = best_fit[~repeated_header_rows]

    best_fit['Source Vertex Index'] = best_fit['Source Vertex Index'].astype(int)
    best_fit['Target Vertex Index'] = best_fit['Target Vertex Index'].astype(int)

    if analysis == 'benson total':
        label_dir = os.path.join(freesurfer, subj, 'label')
        ecc_path = os.path.join(label_dir, f'{hemi}.benson14_eccen-0001.label')
        ang_path = os.path.join(label_dir, f'{hemi}.benson14_angle-0001.label')

        ecc_map = load_label_data(ecc_path)
        ang_map = load_label_data(ang_path)

        best_fit['Source Eccentricity'] = best_fit['Source Vertex Index'].map(ecc_map)
        best_fit['Source Polar Angle'] = best_fit['Source Vertex Index'].map(ang_map)

    else:
        prf_model = filter_prf(prf_params, prf_voxels)
        lh_c = read_morph_data(f'{freesurfer}/{subj}/surf/lh.curv')
        numel_lh = lh_c.shape[0]

        if hemi == 'rh':
            prf_voxels_rh = prf_voxels[prf_voxels >= numel_lh]
            prf_voxels_rh_adjusted = prf_voxels_rh - numel_lh
            prf_ecc_rh = prf_model.ecc[prf_voxels >= numel_lh]
            prf_angle_rh = prf_model.angle[prf_voxels >= numel_lh]
            voxel_to_ecc = dict(zip(prf_voxels_rh_adjusted, prf_ecc_rh))
            voxel_to_angle = dict(zip(prf_voxels_rh_adjusted, prf_angle_rh))
        else:
            voxel_to_ecc = dict(zip(prf_voxels, prf_model.ecc))
            voxel_to_angle = dict(zip(prf_voxels, prf_model.angle))

        best_fit['Source Eccentricity'] = best_fit['Source Vertex Index'].map(voxel_to_ecc)
        best_fit['Source Polar Angle'] = best_fit['Source Vertex Index'].map(voxel_to_angle)

    return best_fit

def header_fix(path, expected_column='Target Vertex Index'):
    with open(path, 'r') as f:
        lines = f.readlines()
    for idx, line in enumerate(lines):
        if expected_column in line:
            header = line
            data = lines[idx + 1:]
            break
    csv_content = ''.join([header] + data)
    return pd.read_csv(io.StringIO(csv_content))

In [4]:
for subj in subjects:
    for atlas in atlases:
        for task in tasks:
            runs = ['']
            if task == 'RestingState':
                runs = ['run-1', 'run-2']

            for run in runs:
                try:
                    prf_params, prf_voxels = load_prf(subj, MAIN_PATH, atlas, denoising, task)
                except Exception as e:
                    print(f"Missing pRF pkl file - Subject: {subj}, Atlas: {atlas}, Task: {task}, Run: {run}: {e}")
                    continue

                for hemi in hemispheres:
                    for target in visual_areas[1:]:  
                        source_target = f"{target}-V1"

                        if task == 'RestingState':
                            best_fit_path = f"{MAIN_PATH}/CFM/{subj}/ses-02/{atlas}/{task}/{run}/{denoising}/GM/{hemi}/{source_target}/best_fits.csv"
                        else:
                            best_fit_path = f"{MAIN_PATH}/CFM/{subj}/ses-02/{atlas}/{task}/{denoising}/GM/{hemi}/{source_target}/best_fits.csv"

                        if not os.path.exists(best_fit_path):
                            print(f'Missing best_fits.csv for {subj}, {atlas}, {target}, {task}, {run}, {hemi}')
                            continue

                        try:
                            best_fit_prf = extract_prf(subj, hemi, target, best_fit_path, prf_params, prf_voxels)
                        except Exception as e:
                            if isinstance(e, KeyError) and 'Source Vertex Index' in str(e):
                                best_fit = header_fix(best_fit_path)
                                best_fit.to_csv(best_fit_path, index=False) 
                                best_fit_prf = extract_prf(subj, hemi, target, best_fit_path, prf_params, prf_voxels)
                            else:
                                print(f'Error processing {subj}, {atlas}, {task}, {run}, {hemi}, {target}: {e}')
                                continue

                        # Save to individual_fits directory
                        output_dir = os.path.join(os.path.dirname(best_fit_path))
                        os.makedirs(output_dir, exist_ok=True)
                        output_path = os.path.join(output_dir, 'best_fits_prf.csv')
                        best_fit_prf.to_csv(output_path, index=False)

                        best_fit_prf.to_csv(output_path, index=False)
                        print(f"[SAVED] best_fits_prf.csv for {subj} | {atlas} | {task} | {run} | {hemi} | {target} here: {output_path}")

                        target_idx = best_fit_prf['Target Vertex Index'].astype(int).values
                        curv = read_morph_data(f'{freesurfer}/{subj}/surf/{hemi}.curv')

                        def write_data_to_surface(values, suffix):
                            masked = np.full(curv.shape[0], 50.0)
                            masked[target_idx] = values
                            output_file = f'{freesurfer}/{subj}/surf/{hemi}.{suffix}_{target}{source}_{denoising}_{task}{("_" + run) if run else ""}_{atlas}'
                            write_morph_data(output_file, masked)

                        write_data_to_surface(best_fit_prf['Source Eccentricity'].values, 'ecc')
                        write_data_to_surface(best_fit_prf['Source Polar Angle'].values, 'pol')
                        write_data_to_surface(best_fit_prf['Best Variance Explained Finer'].values, 've')
                        write_data_to_surface(best_fit_prf['Best Sigma Finer'].values, 'sigma')

[SAVED] best_fits_prf.csv for sub-02 | benson | RET2 |  | lh | V2 here: /Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/CFM/sub-02/ses-02/benson/RET2/nordic/GM/lh/V2-V1/best_fits_prf.csv
[SAVED] best_fits_prf.csv for sub-02 | benson | RET2 |  | lh | V3 here: /Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/CFM/sub-02/ses-02/benson/RET2/nordic/GM/lh/V3-V1/best_fits_prf.csv
[SAVED] best_fits_prf.csv for sub-02 | benson | RET2 |  | rh | V2 here: /Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/CFM/sub-02/ses-02/benson/RET2/nordic/GM/rh/V2-V1/best_fits_prf.csv
[SAVED] best_fits_prf.csv for sub-02 | benson | RET2 |  | rh | V3 here: /Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/CFM/sub-02/ses-02/benson/RET2/nordic/GM/rh/V3-V1/best_fits_prf.csv
[SAVED] best_fits_prf.csv for sub-02 | benson | RET |  | lh | V2 here: /Volumes/FedericaCardillo/pre-processing/projects/PRO